# Import Statements

In [1]:
from __future__ import print_function
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
import enchant
from sklearn.cluster import KMeans
from sklearn import feature_extraction
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import DBSCAN
import numpy as np
from nltk.tokenize import word_tokenize,sent_tokenize

from nltk.stem import WordNetLemmatizer 

import enchant

import os
import time

lemmatizer = WordNetLemmatizer()


# Reading the news articles into a dataframe

In [2]:
df =pd.read_csv("../data/parsed_news_articles.csv")
df['Text'] = df['Text'].astype(str)

# Text Preprocessor

In [3]:
def preprocessor(text):
    stop_words = stopwords.words('english')
    d = enchant.Dict("en_GB")
    spcl_words = ['also','said','olive','glossary','pe','butterfly','lip']
    spcl_names = ['bjp','shaheen','bagh',
                  'caa','nrc','covid','coronavirus','corona','virus','pm']
#     'modi','pm','gandhi','rahul','amit','shah','sonia','congress',
    spcl= '[-,;@_!#$%^&*()<>?/\|}{~:''.+""]'
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens=[]
    for token in tokens:
        token = re.sub(spcl,'',token)
        token = re.sub('[0-9]','',token)
        if re.search('[a-zA-Z]', token):
            token = token.lower()
            if token in spcl_names:
                filtered_tokens.append(token)
            if (token!='') and (d.check(token)) and (token not in spcl_words):
                filtered_tokens.append(token)
    v = [word for word in filtered_tokens if word not in stop_words]
    stems = [lemmatizer.lemmatize(t) for t in v]
    temp = ' '.join(i for i in stems)
    return temp

In [4]:
cleanedArticles=[]

for i in df.index:
    c=preprocessor(df['Text'][i])
    cleanedArticles.append(c)

In [5]:
cleanedArticles[0]

'download time news app latest news subscribe start daily morning time newspaper order welcoming decision prime minister government introduce citizenship amendment act caa national registry citizen nrc urged union government initiate dialogue opposition party dispel doubt controversial cancelled highest decision making body account government advisory cancel mass event due covid scare nonetheless held meeting member allied unit medium resolution passed general secretary indeed troubling opposition party politicising national issue duty union clarify doubt opposition party ruling party prime minister home minister shah initiate dialogue regard functionary quick add previous effort government failed opposition party shown interest understanding need act ruling party initiate dialogue need reciprocated opposition party appear little interest talk meet passed resolution welcoming claiming gave clarity foreigner nrc gave clarity foreigner much needed along caa gave citizenship persecuted mi

In [6]:
df['Cleaned_text']=cleanedArticles

df.to_csv("../data/parsed_cleaned.csv",index=None)


caa_df = df[df['Topic Name']=='caa-nrc']
covid_df = df[df['Topic Name']=='coronavirus']

In [7]:
articles_caa = caa_df['Cleaned_text']
articles_covid = covid_df['Cleaned_text']
vectorizer_caa = TfidfVectorizer()
vectorizer_covid = TfidfVectorizer()
tfidf_matrix_caa = vectorizer_caa.fit_transform(articles_caa)
tfidf_matrix_covid = vectorizer_covid.fit_transform(articles_covid)
print(tfidf_matrix_caa.shape)
print(tfidf_matrix_covid.shape)
terms_caa = vectorizer_caa.get_feature_names()
terms_covid = vectorizer_covid.get_feature_names()

(918, 12054)
(948, 11651)


In [8]:
num_clusters = 3
km_caa = KMeans(n_clusters=num_clusters,max_iter=100)
km_covid = KMeans(n_clusters=num_clusters,max_iter=100)
km_caa.fit(tfidf_matrix_caa)
km_covid.fit(tfidf_matrix_covid)
clusters_caa = km_caa.labels_.tolist()
clusters_covid = km_covid.labels_.tolist()

In [9]:
caa_df['Clusters']=clusters_caa
covid_df['Clusters']=clusters_covid

C:\Users\vogir\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\vogir\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [10]:
print("Top terms per cluster CAA-NRC:")
order_centroids = km_caa.cluster_centers_.argsort()[:, ::-1]
for i in range(num_clusters):
    print("Cluster %d:" % i)
    for ind in order_centroids[i, :10]:
        print(terms_caa[ind])
    print("----------------")
    
    
print("\n\nTop terms per cluster COVID-19:")
order_centroids = km_covid.cluster_centers_.argsort()[:, ::-1]
for i in range(num_clusters):
    print("Cluster %d:" % i)
    for ind in order_centroids[i, :10]:
        print(terms_covid[ind])
    print("----------------")

Top terms per cluster CAA-NRC:
Cluster 0:
police
protest
protester
woman
bagh
student
shaheen
coronavirus
march
people
----------------
Cluster 1:
congress
party
bjp
pm
minister
caa
leader
nrc
government
citizenship
----------------
Cluster 2:
nrc
citizenship
caa
government
resolution
state
country
citizen
minister
act
----------------


Top terms per cluster COVID-19:
Cluster 0:
case
positive
covid
coronavirus
people
tested
state
virus
health
patient
----------------
Cluster 1:
government
virus
coronavirus
minister
people
covid
state
congress
country
worker
----------------
Cluster 2:
pm
light
minister
prime
coronavirus
people
nation
fund
virus
minute
----------------


In [11]:
for i in range(num_clusters):
    articles=[]
    file_name = "../data/caa_nrc_cluster_"+str(i)+".csv"
    for j in caa_df.index:
        
        if caa_df['Clusters'][j]==i:
            title = caa_df['Title'][j]
            title = re.sub('[^\\x00-\\x7F]+','', title)
            article = caa_df['Cleaned_text'][j]
            articles.append([title,article])
    temp = pd.DataFrame(articles,columns=['Title','Text'])
    temp.to_csv(file_name,index=None)
    articles.clear()
    

In [12]:
for i in range(num_clusters):
    articles=[]
    file_name = "../data/covid_cluster_"+str(i)+".csv"
    for j in covid_df.index:
        
        if covid_df['Clusters'][j]==i:
            title = covid_df['Title'][j]
            title = re.sub('[^\\x00-\\x7F]+','', title)
            article = covid_df['Cleaned_text'][j]
            articles.append([title,article])
    temp = pd.DataFrame(articles,columns=['Title','Text'])
    temp.to_csv(file_name,index=None)
    articles.clear()
    